In [ ]:
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_style("darkgrid")

# Select data to plot
df = pd.read_csv(snakemake.input["results"], delimiter=";")
df = df.loc[
    (df["exporter"] == snakemake.wildcards["exporter"])
    & (df["esc"] == snakemake.wildcards["esc"])
    & (df["subcategory"] == "Cost per MWh delivered")
]

df["flexibility"] = df["scenario"].str.replace("_.*", "", regex=True)
df["csp"] = df["scenario"].str.contains("with_csp")

# Add sorted category column to df indicating type of flexibility
# then sort by that category to ensure correct order in x-axis of plot
# Note: Reproduces the order given in "flexibilities".

flexibilities = {
    "unbuffered": "No flexibility",
    "daily": "Daily",
    "weekly": "Weekly",
    "biweekly": "Biweekly",
    "monthly": "Month",
    "quaterly": "Quarterly",
    "annually": "Yearly",
}

df["flexibility"] = df.flexibility.replace(flexibilities)

df["flexibility"] = pd.Categorical(
    df.flexibility, categories=flexibilities.values(), ordered=True
)

df = df.sort_values("flexibility")

# Plot
fig, ax = plt.subplots(figsize=(6, 3))
for label, plot_df in df.groupby("csp"):

    plot_df.plot(
        x="flexibility",
        y="value",
        ax=ax,
        marker="o",
        label="With CSP-TES" if label else "Without CSP-TES",
        fillstyle="full" if label else "none",
    )

ax.set_ylabel("Electricity cost [EUR/MWh]")
ax.set_xlabel("Flexibility to shift load")
ax.set_ylim(0, np.ceil(df["value"].max() / 10) * 10)

ax.legend(loc="lower left")

# Saving figure
fig.savefig(snakemake.output["pdf"], dpi=300, bbox_inches="tight")
fig.savefig(snakemake.output["png"], dpi=300, bbox_inches="tight")